In [ ]:
import os
import cv2 as cv
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from keras import Sequential
from keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

In [ ]:
root = "drive/MyDrive/tgm/resized/"

vuongs = os.listdir(root + "vuongs/")
trons = os.listdir(root + "trons/")
tamgiacs = os.listdir(root + "tamgiacs/")

vuongs_test = os.listdir(root + "test/vuongs/")
trons_test = os.listdir(root + "test/trons/")
tamgiacs_test = os.listdir(root + "test/tamgiacs/")

In [ ]:
vuongs_array = []
trons_array = []
tamgiacs_array = []

kernel = np.ones((7, 7), np.uint8)

for vuong in vuongs:
    img = np.asarray(Image.open(root + "vuongs/" + vuong))
    gray = .2989 * img[:, :, 0] + .5870 * img[:, :, 1] + .1140 * img[:, :, 2]
    gray = 255 - gray
    dilation1 = cv.dilate(gray, kernel, iterations=1)
    dilation2 = cv.dilate(gray, kernel, iterations=2)
    vuongs_array.extend([gray, dilation1, dilation2])

for tron in trons:
    img = np.asarray(Image.open(root + "trons/" + tron))
    gray = .2989 * img[:, :, 0] + .5870 * img[:, :, 1] + .1140 * img[:, :, 2]
    gray = 255 - gray
    dilation1 = cv.dilate(gray, kernel, iterations=1)
    dilation2 = cv.dilate(gray, kernel, iterations=2)
    trons_array.extend([gray, dilation1, dilation2])

for tamgiac in tamgiacs:
    img = np.asarray(Image.open(root + "tamgiacs/" + tamgiac))
    gray = .2989 * img[:, :, 0] + .5870 * img[:, :, 1] + .1140 * img[:, :, 2]
    gray = 255 - gray
    dilation1 = cv.dilate(gray, kernel, iterations=1)
    dilation2 = cv.dilate(gray, kernel, iterations=2)
    tamgiacs_array.extend([gray, dilation1, dilation2])

vuongs_array = np.array(vuongs_array)
trons_array = np.array(trons_array)
tamgiacs_array = np.array(tamgiacs_array)

xtrain = np.concatenate([vuongs_array, trons_array, tamgiacs_array], axis=0)
xtrain = np.expand_dims(xtrain, axis=-1)
print(xtrain.shape)

vuongs_label = np.full(vuongs_array.shape[:1], 0.)
trons_label = np.full(trons_array.shape[:1], 1.)
tamgiacs_label = np.full(tamgiacs_array.shape[:1], 2.)

ytrain = np.concatenate([vuongs_label, trons_label, tamgiacs_label], axis=0)
ytrain.shape

(1080, 60, 60, 1)


(1080,)

In [ ]:
vuongs_array = []
trons_array = []
tamgiacs_array = []

kernel = np.ones((7, 7), np.uint8)

for vuong in vuongs_test:
    img = np.asarray(Image.open(root + "test/vuongs/" + vuong))
    gray = .2989 * img[:, :, 0] + .5870 * img[:, :, 1] + .1140 * img[:, :, 2]
    gray = 255 - gray
    dilation1 = cv.dilate(gray, kernel, iterations=1)
    dilation2 = cv.dilate(gray, kernel, iterations=2)
    vuongs_array.extend([gray, dilation1, dilation2])

for tron in trons_test:
    img = np.asarray(Image.open(root + "test/trons/" + tron))
    gray = .2989 * img[:, :, 0] + .5870 * img[:, :, 1] + .1140 * img[:, :, 2]
    gray = 255 - gray
    dilation1 = cv.dilate(gray, kernel, iterations=1)
    dilation2 = cv.dilate(gray, kernel, iterations=2)
    trons_array.extend([gray, dilation1, dilation2])

for tamgiac in tamgiacs_test:
    img = np.asarray(Image.open(root + "test/tamgiacs/" + tamgiac))
    gray = .2989 * img[:, :, 0] + .5870 * img[:, :, 1] + .1140 * img[:, :, 2]
    gray = 255 - gray
    dilation1 = cv.dilate(gray, kernel, iterations=1)
    dilation2 = cv.dilate(gray, kernel, iterations=2)
    tamgiacs_array.extend([gray, dilation1, dilation2])

vuongs_array = np.array(vuongs_array)
trons_array = np.array(trons_array)
tamgiacs_array = np.array(tamgiacs_array)

xtest = np.concatenate([vuongs_array, trons_array, tamgiacs_array], axis=0)
xtest = np.expand_dims(xtest, axis=-1)
print(xtest.shape)

vuongs_label = np.full(vuongs_array.shape[:1], 0.)
trons_label = np.full(trons_array.shape[:1], 1.)
tamgiacs_label = np.full(tamgiacs_array.shape[:1], 2.)

ytest = np.concatenate([vuongs_label, trons_label, tamgiacs_label], axis=0)
print(ytest.shape)

(180, 60, 60, 1)
(180,)


In [ ]:
xtrain, xval, ytrain, yval = train_test_split(xtrain, ytrain, test_size=.2, random_state=42)

print(xtrain.shape, xval.shape)

y_train = to_categorical(ytrain)
y_val = to_categorical(yval)
y_test = to_categorical(ytest)

train_generator = ImageDataGenerator(rotation_range=5, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=5, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=5, horizontal_flip=True)

train_generator.fit(xtrain)
val_generator.fit(xval)
test_generator.fit(xtest)

input_shape = xtrain[0].shape

(864, 60, 60, 1) (216, 60, 60, 1)


In [ ]:
model = Sequential()

model.add(Flatten(input_shape=input_shape))

model.add(Dense(1028))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(3))
model.add(Activation("softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 3600)              0         
_________________________________________________________________
dense (Dense)                (None, 1028)              3701828   
_________________________________________________________________
activation (Activation)      (None, 1028)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1028)              4112      
_________________________________________________________________
dropout (Dropout)            (None, 1028)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               131712    
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0

In [ ]:
batch_size = 64
epochs = 18

model.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=["accuracy"])

history = model.fit(train_generator.flow(xtrain, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(xval, y_val, batch_size=batch_size), verbose=1)


print(history.history)
print(model.evaluate(xtest, y_test))

Epoch 1/18
14/14 [==============================] - 19s 94ms/step - loss: 1.0965 - accuracy: 0.5982 - val_loss: 4.2096 - val_accuracy: 0.6852
Epoch 2/18
14/14 [==============================] - 1s 42ms/step - loss: 0.6064 - accuracy: 0.7846 - val_loss: 0.3774 - val_accuracy: 0.9306
Epoch 3/18
14/14 [==============================] - 1s 42ms/step - loss: 0.4561 - accuracy: 0.8095 - val_loss: 0.7390 - val_accuracy: 0.8241
Epoch 4/18
14/14 [==============================] - 1s 43ms/step - loss: 0.3737 - accuracy: 0.8497 - val_loss: 0.3559 - val_accuracy: 0.8935
Epoch 5/18
14/14 [==============================] - 1s 43ms/step - loss: 0.3040 - accuracy: 0.8948 - val_loss: 0.2958 - val_accuracy: 0.9120
Epoch 6/18
14/14 [==============================] - 1s 42ms/step - loss: 0.2638 - accuracy: 0.9036 - val_loss: 0.2789 - val_accuracy: 0.9259
Epoch 7/18
14/14 [==============================] - 1s 42ms/step - loss: 0.3357 - accuracy: 0.8745 - val_loss: 0.2305 - val_accuracy: 0.9398
Epoch 8/18
1

In [ ]:
model.save("model_tgm60.h5")